RNN podemos usar, pero is complex to train with longer sequences and can suffer from the problem of vanishing gradients

Hay otra variaciones como LSTM and GRU que no tienen este problema.

In [1]:
import pandas as pd
import scipy.io
import os
import mne
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from datetime import datetime

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking, Embedding
from keras.callbacks import EarlyStopping, ModelCheckpoint

tf.random.set_seed(7)

2023-01-15 02:55:38.620291: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-15 02:55:39.044291: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-15 02:55:39.044309: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-15 02:55:40.137407: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

## Data

### Split files per wildboard and compute the areas points

In [3]:
# USE fiwi for train. becuase other in the positions we dont have too much data

In [4]:
path_files_fiwi = "../Join/telemetriedaten/positions_weather_fiwi/"
# path_files_other = "../Join/telemetriedaten/positions_weather_other/"

path_place_fiwi = "../Join/telemetriedaten/TelemetrieFiwigatter/"
# path_place_other = "../Join/telemetriedaten/TelemetrieJagdgatter/"

save_file_path = "../Join/telemetriedaten/position_per_wild_fiwi/"

In [ ]:
list_files = os.listdir(path_files_fiwi)
wild_boards_fiwi = set()
for file in list_files:
    df = pd.read_csv(path_files_fiwi + file)
    wild_boards_fiwi.update(list(df["ID"].unique()))

# list_files = os.listdir(path_files_other)
# wild_boards_other = set()
# for file in list_files:
#     df = pd.read_csv(path_files_other + file)
#     wild_boards_other.update(list(df["ID"].unique()))

In [5]:
wild_boards_fiwi = {
    'VG.2013.01', 'IS.2011.13', 'FG.2013.16', 'FG.2016.16', 'FG.2013.32', 'FG.2017.12', 'LA.2011.01', 'IS.2011.27',
    'IS.2011.05', 'VG.2013.10', 'IS.2011.20', 'FG.2013.29', 'FG.2016.14', 'FG.2017.11', 'IS.2011.29', 'FG.2016.13', 'IS.2011.10', 
    'IS.2011.37', 'FG.2016.15', 'FG.2016.17', 'FG.2013.09', 'LA.2011.03', 'DE.2011.19', 'IS.2011.21', 'IS.2011.39', 'FG.2017.09', 
    'DE.2011.27', 'IS.2011.32', 'DE.2011.23', 'VG.2013.09', 'FG.2017.18', 'IS.2011.12', 'IS.2011.26', 'IS.2011.25', 'IS.2011.02', 
    'IS.2011.23', 'FG.2016.12', 'FG.2016.03', 'DE.2011.21', 'IS.2011.04', 'IS.2011.11', 'DE.2011.18', 'FG.2016.05', 'FG.2016.07', 
    'FG.2017.16', 'FG.2016.18', 'FG.2017.13', 'IS.2011.22', 'FG.2013.01', 'FG.2013.06', 'VG.2013.02', 'IS.2011.36', 'FG.2017.14', 
    'DE.2011.20', 'FG.2017.17', 'IS.2011.15', 'IS.2011.34', 'DE.2011.14', 'VG.2013.08', 'IS.2011.14', 'DE.2011.25', 'IS.2011.01', 
    'IS.2011.38', 'FG.2013.34', 'FG.2013.25', 'IS.2011.33', 'VG.2013.11'}

In [7]:
for wild in wild_boards_fiwi:
    if not os.path.exists(os.path.join(save_file_path,wild)):
        os.mkdir(os.path.join(save_file_path,wild))

In [8]:
list_files = os.listdir(path_files_fiwi)
for file in list_files:
    df = pd.read_csv(path_files_fiwi + file)
    for wild in wild_boards_fiwi:
        df_to_save = df[df["ID"]==wild]
        df_to_save.to_csv(os.path.join(save_file_path,wild,file))

/tmp/ipykernel_229/2223551429.py:3: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path_files_fiwi + file)
/tmp/ipykernel_229/2223551429.py:3: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path_files_fiwi + file)
/tmp/ipykernel_229/2223551429.py:3: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path_files_fiwi + file)
/tmp/ipykernel_229/2223551429.py:3: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path_files_fiwi + file)
/tmp/ipykernel_229/2223551429.py:3: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path_files_fiwi + file)
/tmp/ipykernel_229/2223551429.py:3: DtypeWarning: Columns (11) have mixed types. Specify dtype optio

In [5]:
# Create file of areas

file_areas = "../Join/telemetriedaten/TelemetrieFiwigatter/TelemetrieFiwigatter_Tel.Fiwi.csv"

df = pd.read_csv(file_areas)
df["start"] = pd.to_datetime(df["start"], infer_datetime_format=True)   
df["end"] = pd.to_datetime(df["end"], infer_datetime_format=True)   

# , "ID" "location","start","end","duration"
# 0,"DE.2011.23","Fiwi",2017-02-01 15:55:08,2017-02-01 15:58:13,3.08333333333333

df_2016 = df[(df["start"] > datetime.strptime("2016-01-01", '%Y-%m-%d')) & (df["start"] < datetime.strptime("2016-12-31", '%Y-%m-%d'))]
df_2017 = df[(df["start"] > datetime.strptime("2017-01-01", '%Y-%m-%d')) & (df["start"] < datetime.strptime("2017-12-31", '%Y-%m-%d'))]
df_2018 = df[(df["start"] > datetime.strptime("2018-01-01", '%Y-%m-%d')) & (df["start"] < datetime.strptime("2018-12-31", '%Y-%m-%d'))]
df_2019 = df[(df["start"] > datetime.strptime("2019-01-01", '%Y-%m-%d')) & (df["start"] < datetime.strptime("2019-12-31", '%Y-%m-%d'))]

In [ ]:
# Unnamed: 0               0.0.5196
# ID                     DE.2011.14
# location                     Fiwi
# start         2017-12-20 13:42:06
# end           2017-12-20 13:42:06
# duration                      0.0
# Name: 78109, dtype: object

df_final = pd.DataFrame(columns=["ID","Date","x-value","y-value","location"])

# for index_data,df in [df_2016,df_2017,df_2018,df_2019]:
for index, row in df.iloc[78100:,:].iterrows():
    id = row["ID"]
    init = row["start"]
    end = row["end"]
    location = row["location"]
    list_dayofcount = [value.strftime("%Y-%m-%d") for value in pd.date_range(start=init,end=end).to_list()]
    for day in list_dayofcount:
        try:
            df_day = pd.read_csv(os.path.join("../Join/telemetriedaten/position_per_wild_fiwi/" + id + "/",day + "_fiwi.csv"))
            df_day["Date"] = pd.to_datetime(df_day["Date"], infer_datetime_format=True)   
            df_to_save = df_day[(df_day["Date"] > init) & (df_day["Date"] < end)][["ID","Date","x-value", "y-value"]]
            df_to_save['location'] = location
            df_final = pd.concat([df_final,df_to_save.iloc[:,:]])
        except Exception as e:
            pass
df_final.to_csv(f"../Join/telemetriedaten/areas.csv")
    # df_final.to_csv(f"../Join/telemetriedaten/areas_{index_data+16}.csv")

In [39]:
df_final.to_csv("../Join/telemetriedaten/areas.csv")

### Fechas juntas

In [ ]:
# Wild board DE.2011.14
df = pd.DataFrame()
for file in os.listdir("../Join/telemetriedaten/position_per_wild_fiwi/DE.2011.14/"):
    print(file)
    df_day = pd.read_csv("../Join/telemetriedaten/position_per_wild_fiwi/DE.2011.14/" + file)
    df_day = df_day[['Date', 'tagid', 'MACadresse', 'x-value', 'y-value', 'ID', 'TempMain', 
                      'HumMain', 'Wind','WindDir', 'Rain', 'Solar', 'TempBB', 'TempForest', 'HumForest',
                      'BaroPressure', '°C_FG', 'hum_FG', 'dew_p_FG', '°C_VG', 'hum_VG', 'dew_p_VG', 'SuhlenVG', 'SuhlenFG']]
    df = pd.concat([df,df_day.iloc[:,:]])
df.to_csv("../Join/telemetriedaten/position_per_wild_fiwi/DE.2011.14/DE.2011.14.csv")

### Prepare files

In [11]:
# Wild board DE.2011.14
path_file = "../Join/telemetriedaten/position_per_wild_fiwi/DE.2011.14/DE.2011.14.csv"
df = pd.read_csv(path_file)
df = df.set_index('Date')
df = df.drop(columns=["Unnamed: 0","tagid","MACadresse","ID"])
ohe = pd.get_dummies(df.WindDir)
df = df.drop('WindDir',axis = 1)
# df = df.join(ohe)

/tmp/ipykernel_467/1405016587.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path_file)


In [20]:
scaler = MinMaxScaler(feature_range=(0,1))
df[['TempMain', 'HumMain', 'Wind', 'Rain', 'Solar',
       'TempBB', 'TempForest', 'HumForest', 'BaroPressure', '°C_FG', 'hum_FG',
       'dew_p_FG', '°C_VG', 'hum_VG', 'dew_p_VG', 'SuhlenVG', 'SuhlenFG']] = scaler.fit_transform(df[['TempMain', 'HumMain', 'Wind', 'Rain', 'Solar',
       'TempBB', 'TempForest', 'HumForest', 'BaroPressure', '°C_FG', 'hum_FG',
       'dew_p_FG', '°C_VG', 'hum_VG', 'dew_p_VG', 'SuhlenVG', 'SuhlenFG']])
values = df.values
values = values.astype('float32')


In [33]:
df_temporal = df.iloc[0:5,:]
df_temporal[["x-value","y-value"]] = df_temporal[["x-value","y-value"]].shift(1)
df_temporal = df_temporal.dropna(subset=["x-value","y-value"])
df_temporal

/tmp/ipykernel_467/1790526574.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temporal[["x-value","y-value"]] = df_temporal[["x-value","y-value"]].shift(1)


,x-value,y-value,TempMain,HumMain,Wind,Rain,Solar,TempBB,TempForest,HumForest,BaroPressure,°C_FG,hum_FG,dew_p_FG,°C_VG,hum_VG,dew_p_VG,SuhlenVG,SuhlenFG
Date,,,,,,,,,,,,,,,,,,,
2017-02-01 17:34:00,0.0,0.0,0.576842,0.253333,0.0,0.0,0.0,0.482201,0.186147,0.931507,0.574899,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-02-01 17:34:00,0.0,0.0,0.576842,0.253333,0.0,0.0,0.0,0.482201,0.186147,0.931507,0.574899,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-02-01 17:36:00,0.0,0.0,0.576842,0.253333,0.0,0.0,0.0,0.482201,0.186147,0.931507,0.574899,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-02-01 17:37:00,0.0,0.0,0.576842,0.253333,0.0,0.0,0.0,0.482201,0.186147,0.931507,0.574899,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Data preparation

In [ ]:
# LSTMs are sensitive to the scale of the input data

# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

In [ ]:
# Split dataset 

train = None
validation = None
test = None

X_train, y_train = None,None
X_validation, y_validation = None,None
X_test, y_test = None,None

## Model and miscelaneous

In [ ]:
...
# design network
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(2))
model.compile(loss='mae', optimizer='adam')
# fit network
history = model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
model = Sequential()

# Embedding layer
model.add(
    Embedding(input_dim=num_words,
              input_length = training_length,
              output_dim=100,
              weights=[embedding_matrix],
              trainable=False,
              mask_zero=True))

# Masking layer for pre-trained embeddings
model.add(Masking(mask_value=0.0))

# Recurrent layer
model.add(LSTM(64, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))

# Fully connected layer
model.add(Dense(64, activation='relu'))

# Dropout for regularization
model.add(Dropout(0.5))

# Output layer
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

## Train model

In [ ]:
# Create callbacks
callbacks = [
                EarlyStopping(monitor='val_loss', patience=5),
                ModelCheckpoint('../models/model.h5', save_best_only=True, save_weights_only=False)
            ]

In [ ]:
history = model.fit(X_train,  y_train, 
                    batch_size=2048, epochs=150,
                    callbacks=callbacks,
                    validation_data=(X_validation, y_validation))

## Results